<a href="https://colab.research.google.com/github/2403A51L33/PfDS-PROJECT/blob/main/ENSEMBLES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import re
import json
import math
import random
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.utils.class_weight import compute_class_weight
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import ComplementNB
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier, StackingClassifier
from sklearn.inspection import permutation_importance
from scipy.stats import randint as sp_randint, uniform as sp_uniform

SEED = 42
random.seed(SEED); np.random.seed(SEED)

DATA_PATH = "/content/realistic_drug_labels_side_effects.csv"
if not os.path.exists(DATA_PATH):
    alt = "./realistic_drug_labels_side_effects.csv"
    if os.path.exists(alt):
        DATA_PATH = alt
    else:
        raise FileNotFoundError(f"CSV not found at {DATA_PATH} or {alt}")

OUTDIR = "./ensemble_outputs"
os.makedirs(OUTDIR, exist_ok=True)

df = pd.read_csv(DATA_PATH)

TEXT_COLS = [c for c in ["indications", "side_effects", "contraindications", "warnings"] if c in df.columns]
NUM_COLS  = [c for c in ["dosage_mg", "price_usd", "approval_year"] if c in df.columns]
CAT_COLS  = [c for c in ["drug_class", "administration_route", "approval_status", "manufacturer"] if c in df.columns]

if len(TEXT_COLS) == 0:
    TEXT_COLS = [c for c in df.columns if df[c].dtype == object and c != "side_effect_severity"]

for c in TEXT_COLS: df[c] = df[c].fillna("")
for c in NUM_COLS:  df[c] = pd.to_numeric(df[c], errors="coerce")
for c in CAT_COLS:  df[c] = df[c].astype(str).fillna("UNK")

def _to_num(v):
    try:
        return float(v)
    except:
        m = {"low":0, "mild":0, "moderate":1, "medium":1, "high":2, "severe":2}
        return m.get(str(v).strip().lower(), np.nan)

y_raw = df["side_effect_severity"]
if y_raw.dtype.kind in "ifu":
    q = np.quantile(y_raw, [0.33, 0.66])
    y = y_raw.apply(lambda v: 0 if v <= q[0] else (1 if v <= q[1] else 2)).astype(int).values
else:
    tmp = y_raw.apply(_to_num)
    if tmp.isna().mean() < 0.5:
        q = np.quantile(tmp.fillna(tmp.median()), [0.33, 0.66])
        y = tmp.fillna(tmp.median()).apply(lambda v: 0 if v <= q[0] else (1 if v <= q[1] else 2)).astype(int).values
    else:
        cats = {k:i for i,k in enumerate(sorted(y_raw.astype(str).unique()))}
        y = y_raw.astype(str).map(cats).values % 3

CLASS_NAMES = ["low","moderate","high"]
NUM_CLASSES = 3

def concat_text_columns(X):
    return X[TEXT_COLS].apply(lambda r: " ".join(map(str, r.values)), axis=1)

text_union = Pipeline([
    ("text_concat", FunctionTransformer(concat_text_columns, validate=False)),
    ("tfidf", TfidfVectorizer(max_features=50000, ngram_range=(1,2), min_df=2))
])

num_proc = Pipeline([
    ("impute", FunctionTransformer(lambda x: np.nan_to_num(x, nan=np.nanmedian(x, axis=0)), accept_sparse=True)),
    ("scale", StandardScaler(with_mean=False))
])

cat_proc = Pipeline([
    ("onehot", OneHotEncoder(handle_unknown="ignore", min_frequency=5))
])

pre = ColumnTransformer(
    transformers=[
        ("text", text_union, TEXT_COLS),
        ("num", num_proc, NUM_COLS) if len(NUM_COLS)>0 else ("num", "drop", []),
        ("cat", cat_proc, CAT_COLS) if len(CAT_COLS)>0 else ("cat", "drop", []),
    ],
    sparse_threshold=0.3
)

X = df[TEXT_COLS + NUM_COLS + CAT_COLS]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y
)

classes = np.unique(y_train)
cw = compute_class_weight(class_weight="balanced", classes=classes, y=y_train)
CLASS_WEIGHTS = {c:w for c,w in zip(classes, cw)}

logreg = Pipeline([
    ("pre", pre),
    ("clf", LogisticRegression(
        class_weight="balanced", solver="saga", max_iter=2000, C=2.0, n_jobs=1, random_state=SEED))
])

svm_base = Pipeline([
    ("pre", pre),
    ("svm", LinearSVC(class_weight="balanced", C=1.0, random_state=SEED))
])
svm = Pipeline([
    ("pre", pre),
    ("cal", CalibratedClassifierCV(estimator=LinearSVC(
        class_weight="balanced", C=1.0, random_state=SEED
    ), method="sigmoid", cv=3))
])

rf = Pipeline([
    ("pre", pre),
    ("clf", RandomForestClassifier(
        n_estimators=400, max_depth=None, min_samples_split=2,
        class_weight="balanced_subsample", n_jobs=-1, random_state=SEED))
])

et = Pipeline([
    ("pre", pre),
    ("clf", ExtraTreesClassifier(
        n_estimators=600, max_depth=None, min_samples_split=2,
        class_weight="balanced", n_jobs=-1, random_state=SEED))
])

cnb = Pipeline([
    ("pre", pre),
    ("clf", ComplementNB(alpha=0.3))
])


BASE_MODELS = {
    "logreg": logreg,
    "svm": svm,
    "rf": rf,
    "et": et,
    "cnb": cnb
}

for name, pipe in BASE_MODELS.items():
    print(f"Training base model: {name}")
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    rep = classification_report(y_test, y_pred, target_names=CLASS_NAMES, output_dict=True, zero_division=0)
    cm = confusion_matrix(y_test, y_pred, labels=[0,1,2]).tolist()
    with open(os.path.join(OUTDIR, f"metrics_{name}.json"), "w") as f:
        json.dump(rep, f, indent=2)
    with open(os.path.join(OUTDIR, f"confusion_matrix_{name}.json"), "w") as f:
        json.dump(cm, f, indent=2)
    print(f"  {name} macro-F1:", rep["macro avg"]["f1-score"])

voters = []
for name in ["logreg","svm","rf","et","cnb"]:
    if name in BASE_MODELS:
        voters.append((name, BASE_MODELS[name].steps[-1][1] if isinstance(BASE_MODELS[name], Pipeline) else BASE_MODELS[name]))

voting = VotingClassifier(
    estimators=[
        ("logreg", logreg),
        ("svm", svm),
        ("rf", rf),
        ("et", et),
        ("cnb", cnb)
    ],
    voting="soft",
    weights=[2,2,2,2,1],
    n_jobs=-1,
    flatten_transform=True
)
param_dist = {
    "weights": [
        [a,b,c,d,e] for a in [1,2,3]
                    for b in [1,2,3]
                    for c in [1,2,3]
                    for d in [1,2,3]
                    for e in [1,2,3]
    ]
}
sampled = random.sample(param_dist["weights"], k=min(25, len(param_dist["weights"])))
search = RandomizedSearchCV(
    voting, param_distributions={"weights": [w for w in sampled]},
    n_iter=len(sampled), scoring="f1_macro", cv=3, n_jobs=-1, random_state=SEED, verbose=0
)
print("Tuning Voting weights...")
search.fit(X_train, y_train)
voting_best = search.best_estimator_
print("Best voting weights:", search.best_params_)

y_pred_v = voting_best.predict(X_test)
rep_v = classification_report(y_test, y_pred_v, target_names=CLASS_NAMES, output_dict=True, zero_division=0)
cm_v = confusion_matrix(y_test, y_pred_v, labels=[0,1,2]).tolist()
with open(os.path.join(OUTDIR, "metrics_voting.json"), "w") as f:
    json.dump(rep_v, f, indent=2)
with open(os.path.join(OUTDIR, "confusion_matrix_voting.json"), "w") as f:
    json.dump(cm_v, f, indent=2)
print("Voting macro-F1:", rep_v["macro avg"]["f1-score"])

stack = StackingClassifier(
    estimators=[
        ("logreg", logreg),
        ("svm", svm),
        ("rf", rf),
        ("et", et),
        ("cnb", cnb)
    ],
    final_estimator=LogisticRegression(solver="lbfgs", max_iter=2000, class_weight="balanced"),
    stack_method="predict_proba",
    cv=5,
    n_jobs=-1,
    passthrough=False
)
print("Training Stacking ensemble...")
stack.fit(X_train, y_train)
y_pred_s = stack.predict(X_test)
rep_s = classification_report(y_test, y_pred_s, target_names=CLASS_NAMES, output_dict=True, zero_division=0)
cm_s = confusion_matrix(y_test, y_pred_s, labels=[0,1,2]).tolist()
with open(os.path.join(OUTDIR, "metrics_stacking.json"), "w") as f:
    json.dump(rep_s, f, indent=2)
with open(os.path.join(OUTDIR, "confusion_matrix_stacking.json"), "w") as f:
    json.dump(cm_s, f, indent=2)
print("Stacking macro-F1:", rep_s["macro avg"]["f1-score"])

def get_feature_names(preprocessor: ColumnTransformer):
    feature_names = []
    for name, transformer, cols in preprocessor.transformers_:
        if name == "remainder":
            continue
        if transformer == "drop":
            continue
        if hasattr(transformer, "named_steps"):
            last = list(transformer.named_steps.values())[-1]

            if isinstance(last, TfidfVectorizer):
                feature_names += [f"text__{t}" for t in last.get_feature_names_out()]

            elif isinstance(last, OneHotEncoder):
                try:
                    ohe = last
                except Exception:

                    for step in transformer.named_steps.values():
                        if isinstance(step, OneHotEncoder):
                            ohe = step; break
                if isinstance(cols, list):
                    col_names = cols
                else:
                    col_names = [c for c in cols]
                ohe_names = list(ohe.get_feature_names_out(col_names))
                feature_names += [f"cat__{t}" for t in ohe_names]
            else:

                if isinstance(cols, list) and len(cols)>0:
                    feature_names += [f"num__{c}" for c in cols]
        else:

            if isinstance(transformer, TfidfVectorizer):
                feature_names += [f"text__{t}" for t in transformer.get_feature_names_out()]
            elif isinstance(transformer, OneHotEncoder):
                ohe_names = list(transformer.get_feature_names_out(cols))
                feature_names += [f"cat__{t}" for t in ohe_names]
            else:
                if isinstance(cols, list) and len(cols)>0:
                    feature_names += [f"num__{c}" for c in cols]
    return np.array(feature_names)

reference_pre = pre

feat_names = None
try:
    pre_fitted = BASE_MODELS["logreg"].named_steps["pre"]
    feat_names = get_feature_names(pre_fitted)
    with open(os.path.join(OUTDIR, "feature_names.txt"), "w", encoding="utf-8") as f:
        for n in feat_names:
            f.write(n + "\n")
except Exception as e:
    print("Feature name extraction failed:", e)

try:
    print("Computing permutation importance for Stacking...")
    pi = permutation_importance(stack, X_test, y_test, n_repeats=10, random_state=SEED, scoring="f1_macro", n_jobs=-1)
    imp = pd.DataFrame({"feature": np.arange(len(pi.importances_mean)),
                        "importance_mean": pi.importances_mean,
                        "importance_std":  pi.importances_std})
    if feat_names is not None and len(feat_names) == imp.shape[0]:
        imp["feature"] = feat_names
    imp.sort_values("importance_mean", ascending=False).to_csv(os.path.join(OUTDIR, "permutation_importance_stacking.csv"), index=False)
except Exception as e:
    print("Permutation importance failed:", e)

def dump_tree_importances(name, pipe):
    try:
        clf = pipe.named_steps["clf"]
        if hasattr(clf, "feature_importances_"):
            pre_fit = pipe.named_steps["pre"]
            names = get_feature_names(pre_fit)
            imp = pd.DataFrame({"feature": names, "importance": clf.feature_importances_})
            imp.sort_values("importance", ascending=False).to_csv(os.path.join(OUTDIR, f"feature_importances_{name}.csv"), index=False)
    except Exception as e:
        print(f"{name} importances failed:", e)

dump_tree_importances("rf", rf)
dump_tree_importances("et", et)

def save_summary(tag, report, cm):
    summary = {
        "macro_f1": report["macro avg"]["f1-score"],
        "weighted_f1": report["weighted avg"]["f1-score"],
        "per_class": {k: report[k] for k in ["low","moderate","high"] if k in report},
        "confusion_matrix": cm
    }
    with open(os.path.join(OUTDIR, f"summary_{tag}.json"), "w") as f:
        json.dump(summary, f, indent=2)

save_summary("voting", rep_v, cm_v)
save_summary("stacking", rep_s, cm_s)

def get_probs(model, X):
    try:
        return model.predict_proba(X)
    except Exception:
        pred = model.predict(X)
        P = np.zeros((len(pred), NUM_CLASSES))
        P[np.arange(len(pred)), pred] = 1.0
        return P

np.save(os.path.join(OUTDIR, "oof_probs_voting.npy"), get_probs(voting_best, X_test))
np.save(os.path.join(OUTDIR, "oof_probs_stacking.npy"), get_probs(stack, X_test))

print("\nDone. Artifacts saved in:", OUTDIR)
print("Top files to check:")
print(" - metrics_.json, confusion_matrix_.json (each base model)")
print(" - metrics_voting.json, metrics_stacking.json")
print(" - permutation_importance_stacking.csv")
print(" - feature_importances_rf.csv / feature_importances_et.csv (if available)")
print(" - feature_names.txt")

Training base model: logreg
  logreg macro-F1: 0.3344953782305226
Training base model: svm
  svm macro-F1: 0.3025114098926853
Training base model: rf
  rf macro-F1: 0.32921622135331813
Training base model: et
  et macro-F1: 0.36134290546055253
Training base model: cnb
  cnb macro-F1: 0.35690058479532166
Tuning Voting weights...
Best voting weights: {'weights': [2, 2, 1, 1, 1]}
Voting macro-F1: 0.3628403394850763
Training Stacking ensemble...
Stacking macro-F1: 0.33994565217391304
Computing permutation importance for Stacking...

Done. Artifacts saved in: ./ensemble_outputs
Top files to check:
 - metrics_.json, confusion_matrix_.json (each base model)
 - metrics_voting.json, metrics_stacking.json
 - permutation_importance_stacking.csv
 - feature_importances_rf.csv / feature_importances_et.csv (if available)
 - feature_names.txt
